# Outline

- Data Processing
- Kmeans clustering
   - 各种传统分类算法
   - PCA
      - SVM（kernel='rbf'）
- Meanshift clustering
   - SVM（kernel='rbf'）
- GMM clustering
   - SVM（kernel='rbf'）
- Spactral Clustering
   - SVM（kernel='rbf'）

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split
from sklearn import *
from scipy import stats
import os
import sys
#import tensorflow_hub as hub
import librosa

In [3]:
import csv
#load data from train_metadata
data = pd.read_csv("/home/yangya/桌面/project/train_metadata.csv/train_metadata.csv",engine='python')
data.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,abethr1,[],['song'],4.3906,38.2788,Turdus tephronotus,African Bare-eyed Thrush,Rolf A. de By,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/128013,abethr1/XC128013.ogg
1,abethr1,[],['call'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363501,abethr1/XC363501.ogg
2,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363502,abethr1/XC363502.ogg
3,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/363503,abethr1/XC363503.ogg
4,abethr1,[],"['call', 'song']",-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/363504,abethr1/XC363504.ogg


In [4]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=101)

In [30]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

# 加载一个音频文件
file_prefix = "/home/yangya/桌面/project/train_audio/"
idx = 5
sample = data.iloc[idx]
path = file_prefix + sample["filename"]
y, sr = librosa.load(path)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

In [6]:
print(mfccs)
print(np.array(mfccs).T.shape)#将计算得到的mfccs保存成转置形式

[[-5.6870575e+02 -4.6347641e+02 -4.3791876e+02 ... -4.5458032e+02
  -4.9969882e+02 -6.2444708e+02]
 [ 9.0777237e+01  8.6492035e+01  7.7351166e+01 ...  4.1411030e+01
   4.3246807e+01  2.6326668e+01]
 [ 2.6606071e+01  2.2994198e+01  1.6594213e+01 ... -2.5461529e+01
  -2.5402603e+01 -1.8658842e+01]
 ...
 [-4.9935865e+00 -3.1236262e+00 -4.9313350e+00 ... -1.2057183e+00
  -2.5710778e+00 -1.4533738e+00]
 [-2.1520042e+00 -2.7340665e+00  6.9367188e-01 ...  6.1726303e+00
   1.7964215e+00 -7.0786185e+00]
 [-2.4106234e-01 -2.8649184e-01  4.6178956e+00 ...  6.1697044e+00
  -1.9447503e+00 -2.4588399e+00]]
(406, 13)


In [7]:
#convert ogg file into mfccs
def convert_to_mfccs(data):
    mfccs_res = []
    file_prefix = "/home/yangya/桌面/project/train_audio/"
    for i in range(len(data)):
        idx =i 
        sample = data.iloc[idx]
        path = file_prefix + sample["filename"]
        y, sr = librosa.load(path)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs_res.append(mfccs.T)
    return mfccs_res

In [8]:
#get the mfccs feature 
train_mfccs = convert_to_mfccs(train_df)
test_mfccs= convert_to_mfccs(test_df)

In [9]:
# compute delta MFCCs
def compute_delta_mfccs(mfccs):
    dmfccs = []
    for m in mfccs:
        tmp = m[1:] - m[0:-1]
        dm = np.hstack((m[0:-1], tmp))
        dmfccs.append(dm)
    return dmfccs

In [10]:
#get the compute delta mfccs
train_dmfccs = compute_delta_mfccs(train_mfccs)
test_dmfccs = compute_delta_mfccs(test_mfccs)

In [11]:
print(np.array(train_dmfccs).shape)

(13552,)


/home/yangya/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [11]:
all_dmfccs = np.vstack(train_dmfccs)
print(all_dmfccs.shape)

(23898718, 26)


# 1.Use clsteruing method to build the Bag-of-'audio' (clustering method:KMeans)

In [13]:
# run k-means to build codebook
km = cluster.KMeans(n_clusters=100, random_state=4487)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

array([[-4.38003296e+02,  4.62914200e+01, -3.30465698e+00, ...,
        -3.51096764e-02, -2.60240138e-01,  3.59313339e-01],
       [-3.84653534e+02,  6.24353294e+01, -3.93464279e+00, ...,
         1.60134047e-01,  6.94406852e-02,  1.87929615e-01],
       [-4.21593597e+02,  3.93832970e+01, -3.12468071e+01, ...,
        -4.15435880e-01, -5.12583196e-01, -1.10433251e-01],
       ...,
       [-3.50888947e+02,  1.01627335e+02, -1.35203018e+01, ...,
        -4.14060801e-02,  1.07255816e-01, -3.94188538e-02],
       [-4.05173096e+02, -9.22915268e+00, -4.27453957e+01, ...,
         1.99975401e-01, -3.31680089e-01,  5.04381321e-02],
       [-1.99721375e+02, -1.10095734e+02, -1.40426117e+02, ...,
         1.44641730e-03,  6.63463119e-03, -2.33691968e-02]], dtype=float32)

In [14]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

In [15]:

train_bow = bow_transform(km, train_dmfccs)
test_bow = bow_transform(km,test_dmfccs)

In [16]:
print(train_bow.shape)

(13552, 100)


In [16]:
tagnames = data["primary_label"].unique()

In [17]:
# convert list of tags into binary class labels
def tags2class(tags, tagnames):
    b = np.zeros(shape=(len(tags), len(tagnames)))
    for i,t in enumerate(tags):
        for j,n in enumerate(tagnames):
            if n in t:
                b[i,j] = 1
    return b

In [18]:
# train_classes[i,j] = absence/presence of the j-th tag in the i-th sound
train_classes_ = tags2class(train_df['primary_label'], tagnames)
test_classes_ = tags2class(test_df['primary_label'], tagnames)

In [19]:
train_classes = []
for i in range(len(train_classes_)):
    train_classes.append(np.argmax(train_classes_[i]))

In [20]:
test_classes = []
for j in range(len(test_classes_)):
    test_classes.append(np.argmax(test_classes_[j]))

In [43]:
print(np.array(train_classes).shape)
print(np.array(test_classes).shape)
#print(test_classes_)

(13552,)
(3389,)


# Experiment

In [21]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
train_Xtf = tf_trans.fit_transform(train_bow)
test_Xtf  = tf_trans.transform(test_bow)


## Naive Bayes Multinomial TF-IDF with BOW

In [ ]:
# Train Multinomial NB model
def trainMModel(a):
    mmodel = naive_bayes.MultinomialNB(alpha = a)
    mmodel.fit(train_bow,train_classes)
    
    testY = test_classes
    predY = mmodel.predict(test_bow)
    acc = metrics.accuracy_score(testY, predY)
    
    return acc

# Grid Search to find the best performance parameter setting (alpha)
def GridSearchMModel(start_a, num_a):
    best_a = start_a
    best_acc = 0.0
    for i in range(0, num_a):
        a = start_a + (i / (num_a - 1))
        tmp_acc = trainMModel(a)
        if tmp_acc > best_acc:
            best_a = a
            best_acc = tmp_acc
    return best_a, best_acc

best_a, best_acc = GridSearchMModel(0.0, 10001)
print("Parameter setting with best performance: alpha = {}, accuracy = {}".format(best_a, best_acc))

## Naive Bayes Multinomial TF-IDF with TF-IDF

In [44]:
# Train Multinomial NB model
def trainMModel(a):
    mmodel = naive_bayes.MultinomialNB(alpha = a)
    mmodel.fit(train_Xtf,train_classes)
    
    testY = test_classes
    predY = mmodel.predict(test_Xtf)
    acc = metrics.accuracy_score(testY, predY)
    
    return acc

# Grid Search to find the best performance parameter setting (alpha)
def GridSearchMModel(start_a, num_a):
    best_a = start_a
    best_acc = 0.0
    for i in range(0, num_a):
        a = start_a + (i / (num_a - 1))
        tmp_acc = trainMModel(a)
        if tmp_acc > best_acc:
            best_a = a
            best_acc = tmp_acc
    return best_a, best_acc

best_a, best_acc = GridSearchMModel(0.0, 10001)
print("Parameter setting with best performance: alpha = {}, accuracy = {}".format(best_a, best_acc))

/home/yangya/anaconda3/envs/ML/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


Parameter setting with best performance: alpha = 0.046, accuracy = 0.08291531425199174


## TF-IDF is better than BOW, so in the next experiments, we use TF-IDF as features to classify species

## Suport Vector Machine(SVM) using linear kernel

In [ ]:
# Using SVM with linear kernel
def trainSVM(c):
    clf = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', svm.SVC(C = c, kernel = 'linear'))])
    svm_clf = clf.fit(train_Xtf, train_classes)
    svm_predY = svm_clf.predict(test_Xtf)
    acc_svm = metrics.accuracy_score(test_classes, svm_predY)
    
    return acc_svm

# Grid Search to find the C with best performance
def GridSearchSVM(Cs):
    best_c = Cs[0]
    best_acc = 0.0
    for i in range(len(Cs)):
        tmp_acc = trainSVM(Cs[i])
        if tmp_acc > best_acc:
            best_c = Cs[i]
            best_acc = tmp_acc
    return best_c, best_acc

Cs = np.logspace(-5, 5, 50)
best_c, best_acc_svm = GridSearchSVM(Cs)
print("Parameter setting with best performance: C = {}, accuracy = {}".format(best_c, best_acc_svm))

## Support Vector Machine (SVM) using linear kernel with cross-validation

In [48]:
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv = model_selection.GridSearchCV(svm.SVC(kernel = 'linear'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv.fit(train_Xtf , train_classes);

print("best params:", svmcv.best_params_)
# predict from the model
predY1 = svmcv.best_estimator_.predict(test_Xtf)

# calculate accuracy
acc1 = metrics.accuracy_score(test_classes, predY1)
print("test accuracy =", acc1)


{'C': array([1.00000000e-05, 1.59985872e-05, 2.55954792e-05, 4.09491506e-05,
       6.55128557e-05, 1.04811313e-04, 1.67683294e-04, 2.68269580e-04,
       4.29193426e-04, 6.86648845e-04, 1.09854114e-03, 1.75751062e-03,
       2.81176870e-03, 4.49843267e-03, 7.19685673e-03, 1.15139540e-02,
       1.84206997e-02, 2.94705170e-02, 4.71486636e-02, 7.54312006e-02,
       1.20679264e-01, 1.93069773e-01, 3.08884360e-01, 4.94171336e-01,
       7.90604321e-01, 1.26485522e+00, 2.02358965e+00, 3.23745754e+00,
       5.17947468e+00, 8.28642773e+00, 1.32571137e+01, 2.12095089e+01,
       3.39322177e+01, 5.42867544e+01, 8.68511374e+01, 1.38949549e+02,
       2.22299648e+02, 3.55648031e+02, 5.68986603e+02, 9.10298178e+02,
       1.45634848e+03, 2.32995181e+03, 3.72759372e+03, 5.96362332e+03,
       9.54095476e+03, 1.52641797e+04, 2.44205309e+04, 3.90693994e+04,
       6.25055193e+04, 1.00000000e+05])}
Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/yangya/anaconda3/envs/ML/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


: 

: 

##  Support Vector Machine (SVM) using RBF kernel

In [ ]:
# Using SVM with linear kernel
def trainSVMRBF(c):
    clf = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', svm.SVC(C = c, kernel = 'rbf'))])
    svm_clf = clf.fit(train_Xtf, train_classes)
    svm_predY = svm_clf.predict(test_Xtf)
    acc_svm = metrics.accuracy_score(test_classes, svm_predY)
    
    return acc_svm

# Grid Search to find the C with best performance
def GridSearchSVMRBF(Cs):
    best_c = Cs[0]
    best_acc = 0.0
    for i in range(len(Cs)):
        tmp_acc = trainSVMRBF(Cs[i])
        if tmp_acc > best_acc:
            best_c = Cs[i]
            best_acc = tmp_acc
    return best_c, best_acc

Cs = np.logspace(-5, 5, 50)
best_c3, best_acc_svm3 = GridSearchSVMRBF(Cs)
print("Parameter setting with best performance: C = {}, accuracy = {}".format(best_c3, best_acc_svm3))

## Support Vector Machine (SVM) using RBF kernel with cross-validation

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(train_Xtf, train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(test_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

## Gradient Boosting

In [ ]:
import xgboost as xgb
# Gradient Boosting

paramsampler= {    
    "colsample_bytree": stats.uniform(0.7, 0.3),  
    "gamma":            stats.uniform(0, 0.5),    
    "max_depth":        stats.randint(2, 6),      
    "subsample":        stats.uniform(0.6, 0.4),  
    "learning_rate":    stats.uniform(.001,1),    
    "n_estimators":     stats.randint(10, 1000),
}

#X_train, X_test, y_train, y_test = model_selection.train_test_split(trainXtf, trainY, test_size = 0.2, random_state = 0)
xclf = xgb.XGBClassifier(objective = "multi:softmax", random_state = 4487)
xgbcv = model_selection.RandomizedSearchCV(xclf, param_distributions = paramsampler, random_state = 4487, n_iter = 200, cv = 5, verbose = 1, n_jobs = -1)
xgbcv.fit(train_Xtf , train_classes)

print("best params:", xgbcv.best_params_)

In [ ]:
xgb_predY = xgbcv.best_estimator_.predict(test_Xtf)
acc_xgb = metrics.accuracy_score(test_classes, xgb_predY)
print("Gradient Boosting classifier accuracy: {}".format(acc_xgb))

## Random Forest

In [ ]:
paramsampler = {#'max_features': stats.uniform(0,1.0),
                 'max_depth':         stats.randint(1,5),
                 'min_samples_split': stats.uniform(0,0.5), 
                 'min_samples_leaf':  stats.uniform(0,0.5),
               }

rfrcv = model_selection.RandomizedSearchCV(
                            ensemble.RandomForestClassifier(n_estimators = 100, random_state = 4487, n_jobs = -1),
                            param_distributions = paramsampler, 
                            random_state = 4487, n_iter = 1000, cv = 5, 
                            verbose = 1, n_jobs = -1)

rfrcv.fit(train_Xtf, train_classes);

print("best params:", rfrcv.best_params_)

In [ ]:
rf_predY = rfrcv.best_estimator_.predict(test_Xtf)
acc_rf = metrics.accuracy_score(test_classes, rf_predY)
print("Random Forest classifier accuracy: {}".format(acc_rf))

## Logistic Regression

In [ ]:
# Using LR
def trainLR(c):
    logreg = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', linear_model.LogisticRegression(n_jobs = -1, C = c, solver = 'saga'))])
    lr_clf = logreg.fit(train_Xtf, train_classes)
    lr_predY = lr_clf.predict(test_Xtf)
    acc_lr = metrics.accuracy_score(test_classes, lr_predY)
    
    return acc_lr

# Grid Search to find the C with best performance
def GridSearchLR(Cs):
    best_c = Cs[0]
    best_acc = 0.0
    for i in range(len(Cs)):
        tmp_acc = trainLR(Cs[i])
        if tmp_acc > best_acc:
            best_c = Cs[i]
            best_acc = tmp_acc
    return best_c, best_acc

Cs = logspace(-5, 5, 50)
best_c_lr, best_acc_lr = GridSearchLR(Cs)
print("Parameter setting with best performance: C = {}, accuracy = {}".format(best_c_lr, best_acc_lr))

## Logistic Regression with cross-validation

In [ ]:
Cs = logspace(-5, 5, 50)

# setup the cross-validation object
lrcv_clf = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', linear_model.LogisticRegressionCV(n_jobs = -1, Cs = Cs, solver = 'saga', max_iter = 10000))])
lrcv = lrcv_clf.fit(train_Xtf, train_classes)

lr_predY = lrcv.predict(test_Xtf)
acc_lrcv = metrics.accuracy_score(test_classes, lr_predY)

print("Accuracy of LR model with cross-validation: ", acc_lrcv)

# Use PCA to reduce dimensionality

In [40]:
pca_model = decomposition.TruncatedSVD(n_components=90)
pca_model.fit(train_Xtf)

TruncatedSVD(n_components=90)

In [41]:
import matplotlib.pyplot as plt

In [44]:
# This function plot the PCA curve
def plot_exp_ratio(ratio, title):
    explain_fig = plt.figure()
    idx = np.where(ratio > 0.95)[0]#[0]
    print("95% ratio when components are {}".format(idx))
    #plt.title(title)
    #plt.plot(ratio)

In [ ]:
#print((np.cumsum(pca_model.explained_variance_ratio_)).shape)
plot_exp_ratio(np.cumsum(pca_model.explained_variance_ratio_), 
               "Explained Variance Ratio(PCA)")

### Covert the tran_Xtf and test_xtf to reduced dimensionality

In [48]:
pca_500To285 = decomposition.TruncatedSVD(n_components=76)
pca_500To285.fit(train_Xtf)

train_Xtfpca = pca_500To285.transform(train_Xtf)
test_Xtfpca = pca_500To285.transform(test_Xtf)

In [49]:
print(train_Xtfpca.shape)
print(test_Xtfpca.shape)

(13552, 76)
(3389, 76)


## Use SVM (kernel='rbf') to estimate [features dimensionality reduced by PCA]

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(train_Xtfpca, train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(test_Xtfpca)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

# 2.Use clsteruing method to build the Bag-of-'audio' (clustering method:Mean-shift)

In [53]:
# run k-means to build codebook
km = cluster.MeanShift(bandwidth=5, bin_seeding=True, n_jobs=-1)#cluster.KMeans(n_clusters=100, random_state=4487)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

array([[-6.9485876e+02,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-5.8202631e+02,  0.0000000e+00,  0.0000000e+00, ...,
         2.8103402e-02,  2.8199553e-02,  2.9092280e-02],
       [-6.5329901e+02, -2.7558545e-02, -3.7574891e-03, ...,
        -1.2676661e-02, -2.2265969e-02, -2.8293958e-02],
       ...,
       [-8.6369983e+02,  7.1583819e-01, -7.5337625e-01, ...,
        -8.6526990e-02, -6.8813443e-01, -8.1724358e-01],
       [-8.8347571e+02,  7.0363402e-02,  5.7839528e-02, ...,
        -2.8870828e+00, -3.5360148e+00, -4.3775770e-01],
       [-8.8555505e+02,  5.6913698e-01, -4.2579567e-01, ...,
         8.0586481e-01,  2.9934841e-01,  1.6970888e-01]], dtype=float32)

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

trainmeanshift_bow = bow_transform(km, train_dmfccs)
testmeanshift_bow = bow_transform(km,test_dmfccs)

## SVM(kernel='rbf') [features extracted by Meanshift method]

In [ ]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
trainmeanshift_Xtf = tf_trans.fit_transform(trainmeanshift_bow )
testmeanshift_Xtf  = tf_trans.transform(testmeanshift_bow)

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainmeanshift_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testmeanshift_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

# 3.Use clsteruing method to build the Bag-of-'audio' (clustering method:GMM)

In [ ]:
# run k-means to build codebook
km = mixture.GaussianMixture(n_components=100, covariance_type='full', random_state=4487, n_init=10)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

trainGMM_bow = bow_transform(km, train_dmfccs)
testGMM_bow = bow_transform(km,test_dmfccs)

## SVM(kernel='rbf') [features extracted by GMM method]

In [ ]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
trainGMM_Xtf = tf_trans.fit_transform(trainGMM_bow )
testGMM_Xtf  = tf_trans.transform(testGMM_bow)

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainGMM_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testGMM_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

# 4.Use clsteruing method to build the Bag-of-'audio' (clustering method:Spectral Clustering)

In [ ]:
# run k-means to build codebook
km = luster.SpectralClustering(n_clusters=4100, affinity='rbf', gamma=1.0, assign_labels='discretize', n_jobs=-1)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

trainS_bow = bow_transform(km, train_dmfccs)
testS_bow = bow_transform(km,test_dmfccs)

## SVM(kernel='rbf') [features extracted by Spectral clustering method]

In [ ]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
trainS_Xtf = tf_trans.fit_transform(trainS_bow )
testS_Xtf  = tf_trans.transform(testS_bow)

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainS_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testS_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)